In [2]:
import os
import glob
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm

In [3]:
# 2. Definir rutas del dataset

train_path = '/kaggle/input/covid19-image-dataset/Covid19-dataset/train/'

labels = os.listdir(train_path)
print(f"Clases encontradas: {labels}")

Clases encontradas: ['Normal', 'Viral Pneumonia', 'Covid']


In [4]:
# 3. Cargar y preparar las imagenes 

X = []
y = []

for label_int, label_string in enumerate(labels):
    path = os.path.join(train_path, label_string)
    image_files = glob.glob(path + "/*.jpg") + glob.glob(path + "/*.jpeg") + glob.glob(path + "/*.png")

    for img_file in image_files: 
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        X.append(img)
        y.append(label_int)

X = np.array(X)
y = np.array(y)

X = preprocess_input(X)

print(f"Forma final de X: {X.shape}")

Forma final de X: (251, 224, 224, 3)


In [5]:
# 4. Dividir data en entrenamiento y validacion

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Conjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de entrenamiento: {X_val.shape}")

Conjunto de entrenamiento: (200, 224, 224, 3)
Conjunto de entrenamiento: (51, 224, 224, 3)


In [7]:
# 5. Crear el modelo Transfer Learning

# Cargar MobileNetV2
base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 3))

# Freezear las capas base
base_model.trainable = False

# Input
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

# Compilar modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
# 6. Entrenamiento del Modelo

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32
)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3975 - loss: 1.0976 - val_accuracy: 0.4510 - val_loss: 1.0946
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 862ms/step - accuracy: 0.4504 - loss: 1.0941 - val_accuracy: 0.4510 - val_loss: 1.0919
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 836ms/step - accuracy: 0.4544 - loss: 1.0913 - val_accuracy: 0.4510 - val_loss: 1.0896
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 865ms/step - accuracy: 0.4502 - loss: 1.0893 - val_accuracy: 0.4510 - val_loss: 1.0871
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 859ms/step - accuracy: 0.4129 - loss: 1.0905 - val_accuracy: 0.4510 - val_loss: 1.0845
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 862ms/step - accuracy: 0.4512 - loss: 1.0840 - val_accuracy: 0.4510 - val_loss: 1.0815
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 841ms/step - accuracy: 0.4536 - loss: 1.0804 - val_accuracy: 0.4510 - val_loss: 1.0785
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 834ms/step - accuracy: 0.4432 - loss: 1.0801 - val_accuracy: 0.4510 - val